# Cyber Research Latest Reports from 2024-2025

In this agentic AI, I automated the process to collect latest reports prepared by companies from 2024 to 2025. I used crewai to create it.  

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI library.

In [3]:
from crewai import Agent, Task, Crew, Process

- As a LLM for your agents, I'll be using OpenAI's `gpt-4o-mini`.



In [ ]:
import os
from crewai_tools import SerperDevTool
from dotenv import load_dotenv

load_dotenv() 

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini' 

search_tool = SerperDevTool()


## Creating Agents

- Define my Agents, and provide them a `role`, `goal` and `backstory`.


In [5]:
researcher = Agent(
    role='Cyber Security Researcher',
    goal='Find the 15 latest cyber attack reports in PDF format from late 2024 and 2025.',
    backstory="""You are an expert in cyber threat intelligence. You specialize 
    in finding formal technical whitepapers and incident reports from 
    reputable security firms (like Mandiant, CrowdStrike, or CISA).""",
    tools=[search_tool],
    verbose=True,
    allow_delegation=False
)

### Agent: Writer

In [6]:
writer = Agent(
    role='Security Technical Analyst',
    goal='Summarize technical reports and verify PDF links.',
    backstory="""You excel at distilling complex attack vectors into brief, 
    actionable summaries. You ensure that every link provided leads 
    directly to a PDF document.""",
    verbose=True,
    allow_delegation=False
)

In [7]:
editor = Agent(
    role='Senior Security Editor',
    goal='Final review and polish of the report',
    backstory='Expert at ensuring technical accuracy and professional tone.',
    verbose=True
)

### Agent: Editor

In [8]:
research_task = Task(
    description="""Search the internet for the 15 most recent technical reports 
    on new attack types (e.g., AI-driven phishing, ransomware variants). 
    Ensure you specifically look for PDF files using search queries 
    like 'filetype:pdf latest cyber attack reports 2025'.""",
    expected_output="A list of 15 unique PDF URLs and their titles.",
    agent=researcher
)

## Creating Tasks

- Define my Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [9]:
research_task = Task(
    description=(
        "1. Search for the 15 most recent technical cyber security reports "
        "released in late 2024 or 2025 focusing on new attack types.\n"
        "2. Specifically target file types like PDF from reputable sources "
        "such as CISA, Mandiant, CrowdStrike, and SANS Institute.\n"
        "3. For each report, extract the exact direct URL to the PDF file.\n"
        "4. Filter out outdated or non-technical news articles."
    ),
    expected_output=(
        "A list of 15 unique entries containing the report title, "
        "the publishing organization, and the direct PDF download link."
    ),
    agent=researcher,
)

### Task: Write

In [ ]:
reporting_task = Task(
    description=(
        "1. Review the 15 PDF links provided by the researcher.\n"
        "2. For each report, write a brief summary (2-3 sentences) "
        "explaining the specific attack vector or threat discussed.\n"
        "3. Ensure the final document is professionally formatted in Markdown.\n"
        "4. Each entry must clearly display the 'Brief Summary' and 'PDF Link'."
    ),
    expected_output=(
        "A comprehensive Markdown report summarizing the 15 latest "
        "cyber attack trends with clickable PDF resources for further reading."
    ),
    agent=writer,
    context=[research_task],
)

### Task: Edit

In [ ]:
edit_task = Task(
    description=(
        "1. Proofread the generated cyber security report for technical accuracy "
        "and grammatical errors.\n"
        "2. Ensure the tone is professional, authoritative, and aligned with "
        "cyber threat intelligence standards.\n"
        "3. Check that all 15 PDF links are active and correctly formatted.\n"
        "4. Verify that each attack type summary is concise and easy to understand."
    ),
    expected_output=(
        "A polished, final version of the cyber security report in Markdown format. "
        "The report should be ready for professional distribution, with each summary "
        "neatly structured and limited to 2 or 3 clear paragraphs."
    ),
    agent=editor,
    context=[reporting_task],
)

## Creating the Crew

- Create my crew of Agents


In [ ]:
cyber_crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, reporting_task, edit_task],
    process=Process.sequential,
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for the same input, so what you get might be different than what I have here. 

In [ ]:
# Latest Cyber Attack Trends Report

result = cyber_crew.kickoff()

- Display the results of my execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)